In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import pandas as pd
from pathlib import Path
from deltalake import DeltaTable
import src.config
import src.sba_loans.fetch_data
import cdutils.all_status_silver_acct.core

# def main_report_creation():
# Get data from custom silver table that includes all accounts 
# Reason is that the other one is only looking at active accounts
df = cdutils.all_status_silver_acct.core.query_df_on_date()

# Get Lakehouse tables
# address = DeltaTable(src.config.BRONZE / "wh_addr").to_pandas()

orgpersaddr_data = src.sba_loans.fetch_data.fetch_org_pers_addr()
orgaddruse = orgpersaddr_data['orgaddruse'].copy()
persaddruse = orgpersaddr_data['persaddruse'].copy()
wh_addr = orgpersaddr_data['wh_addr'].copy()

# Filter to SBA loans and select relevant columns
df = df[df['product'].str.contains('SBA', case=False, na=False)].copy()
df = df[[
    'acctnbr',
    'ownersortname',
    'portfolio_key',
    'product',
    'mjaccttypcd',
    'currmiaccttypcd',
    'curracctstatcd',
    'loanofficer',
    'taxrptfororgnbr',
    'taxrptforpersnbr'
]].copy()



# 
orgaddruse = orgaddruse[orgaddruse['addrusecd'].isin(['PRI'])].copy()
persaddruse = persaddruse[persaddruse['addrusecd'].isin(['PRI'])].copy()


# %%
orgaddruse['orgnbr'] = orgaddruse['orgnbr'].astype(str)
orgaddruse['addrnbr'] = orgaddruse['addrnbr'].astype(str)
persaddruse['persnbr'] = persaddruse['persnbr'].astype(str)
persaddruse['addrnbr'] = persaddruse['addrnbr'].astype(str)


# return final_df


In [ ]:
wh_addr = wh_addr[[
    'addrnbr',
    'text1',
    'text2',
    'text3',
    'text4',
    'text5',
    'cityname',
    'statecd',
    'zipcd'
]].copy()

In [ ]:
df.info()

In [ ]:
import numpy as np
df['taxrptfororgnbr'] = np.where(df['taxrptfororgnbr'].isna(), np.nan, df['taxrptfororgnbr'].astype('Int64').astype(str))
df['taxrptforpersnbr'] = np.where(df['taxrptforpersnbr'].isna(), np.nan, df['taxrptforpersnbr'].astype('Int64').astype(str))



In [ ]:
df

In [ ]:
orgaddruse = orgaddruse[[
    'orgnbr',
    'addrnbr'
]].copy()

In [ ]:
persaddruse = persaddruse[[
    'persnbr',
    'addrnbr'
]].copy()

In [ ]:
wh_addr

In [ ]:
orgaddruse.info()

In [ ]:
wh_addr['addrnbr'] = wh_addr['addrnbr'].astype(str)

In [ ]:
orgaddruse = orgaddruse.merge(wh_addr, on='addrnbr', how='inner')

In [ ]:
orgaddruse

In [ ]:
df = df.merge(orgaddruse, left_on='taxrptfororgnbr', right_on='orgnbr', how='left')

In [ ]:
df

In [ ]:
df = df.drop(columns=['taxrptfororgnbr','orgnbr','addrnbr']).copy()

In [ ]:
df

In [ ]:
persaddruse = persaddruse.merge(wh_addr, on='addrnbr', how='inner')

In [ ]:
persaddruse

In [ ]:
df = df.merge(persaddruse, left_on='taxrptforpersnbr', right_on='persnbr', how='left')

In [ ]:
df

In [ ]:
df = df.drop(columns=['taxrptforpersnbr','addrnbr','persnbr']).copy()

In [ ]:
df

In [ ]:
df_copy = df.copy()

In [ ]:
base_names = {col[:-2] for col in df_copy.columns if col.endswith('_x')}

for base in base_names:
    col_x = f"{base}_x"
    col_y = f"{base}_y"

    if col_y in df_copy.columns:
        df_copy[base] = df_copy[col_x].fillna(df_copy[col_y])
        df_copy = df_copy.drop(columns=[col_x, col_y])

In [ ]:
df_copy

In [ ]:
final_df = df_copy[[
    'acctnbr',
    'ownersortname',
    'portfolio_key',
    'product',
    'mjaccttypcd',
    'currmiaccttypcd',
    'curracctstatcd',
    'loanofficer',
    'text1',
    'text2',
    'text3',
    'text4',
    'text5',
    'cityname',
    'statecd',
    'zipcd'
]].copy()

In [ ]:
final_df

In [ ]:
final_df['product'].unique()

In [ ]:
final_df